In [ ]:
import pandas as pd
from pyproj import Transformer #Für Umrechnen der Koordinaten

import regex as re

#!pip install geopy #für das Abfragen ...
import geopy
from geopy.geocoders import GoogleV3 #der Google-API (Addressabfrage)

from geopy.extra.rate_limiter import RateLimiter #Verzögern der Abfrage

## 1. Koordinaten umrechnen
Von EPSG2056 (LV95) zu EPSG4326 (WSG84)

https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

In [ ]:
transformer = Transformer.from_crs("epsg:2056", "epsg:4326")

transformer.transform(2743846.709100001, 1259012.603399999)

#Test, Überprüfen mit Tool von swisstopo (https://www.swisstopo.admin.ch/en/maps-data-online/calculation-services/navref.html)
#Ergebnis: kleine Abweichung, aber auf der Karte dasselbe Erlebnis


In [ ]:
df = pd.read_csv('daten/larmemission-streetFULL.csv')

In [ ]:
df.tail(2)

In [ ]:
df.shape

In [ ]:
transformer.transform(df['Coordinates'][6])

In [ ]:
gps = []

for i in range(len(df)): #liest Spalten latitude und longitude aus,
    #transformiert diese in EPSG4326
    r = transformer.transform(df['latitude'][i], df['longitude'][i])
    
    gps.append(r) #packt die Ergebnisse in die List gps

In [ ]:
gps[10][0]

In [ ]:
#fügt dem df die Spalte GPS mit dem Inhalt der List GPS an
df['gps'] = gps

In [ ]:
df.head(3)

In [ ]:
# Zwischenergebnis in einem csv abspeichern
df.to_csv(r'daten/V3_larmemissionen-strasse-mit gps.csv', index = False)

## 2. Koordinaten filtern

Alles südlich der Stadt St.Gallen kann weg, also alle Daten, deren Latitude tiefer ist als 47.395407 (bzw. tiefer als 1249895.35).

Dadurch behalte ich die Daten zu den weiteren Städten (Gossau, Wil, Rorschach) weiterhin. Diese könnten für weitere Auswertungen (weitere Geschichten) interessant sein.

--> nur behalten, was >= 1249895

In [ ]:
df_sg = df[df['longitude']>= 1249895]

In [ ]:
df_sg.head(2)

In [ ]:
#Durch den Filter fallen rund 10'000 rows weg
df_sg.shape

In [ ]:
# Zwischenergebnis abspeichern
df_sg.to_csv('daten/V3_df-sg.csv', index = False)

## 3. Aus den Koordinaten via Google-API Adressen ziehen

In [ ]:
df_sg.dtypes

In [ ]:
df_sg.head(3)

In [ ]:
df_sg.tail(5)

In [ ]:
df_sg.shape

In [ ]:
# Google-API geocoding als gelocater setzen (API-Key mitgeben)
geolocator = GoogleV3(api_key = 'XXXX')

In [ ]:
#RateLlimiter auf zwei Sekunden setzen
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)


addresses = []

for i in range(len(df_sg)):
    #speichert aufgrund der Koordinaten das Adress-File
    location = geolocator.reverse(df_sg['gps'].iloc[i])
    
    #holt aus dem Adressen-File die raw-Daten
    value = location.address
    
    #packt die raw-Daten in eine Liste
    addresses.append(value)

In [ ]:
addresses

In [ ]:
#mit Regex den Leerschlag zwischen St. und Strassennamen oder Gemeindenamen entfernen
address_new = [re.sub(r'St. ', 'St.', i) for i in addresses]

In [ ]:
df_sg = df_sg.copy()

In [ ]:
#Addressen dem df2 hinzufügen
df_sg['addresses'] = address_new

In [ ]:
df_sg.head(12)

In [ ]:
df_sg.to_csv('daten/sg_with addresses_uncleaned.csv', index = False)

### Adressen aufsplitten in Strasse, Gemeinde, PLZ 

In [ ]:
# Adressen beim Komma splitten, um Strassen und Gemeindenamen seperat zu erhalten
address_list = []

for i in range(len(df_sg)):
    
    elem = df_sg['addresses'].iloc[i].split(',')
    
    address_list.append(elem)

In [ ]:
#mit for-loop Strassen und Gemeinden aus der Liste auslesen und in seperate Listen packen
strasse = []
gemeinde = []

for i in range(len(address_list)):
    elem_str = address_list[i][0]
    strasse.append(elem_str)
    
    elem_gem = address_list[i][1]
    gemeinde.append(elem_gem)

In [ ]:
#Strasse und Gemeinde dem df hinzufügen
df_sg['strasse'] = strasse
df_sg['gemeinde'] = gemeinde

In [ ]:
df_sg.to_csv('daten/df_sg_with addresses_2.csv', index = False)

#### Spalte mit Adresse bzw. Listen bereinigen

In [ ]:
strasse

In [ ]:
gemeinde

In [ ]:
gemeinde
#-> mit Regex alle rausfiltern, die am Anfang nicht das Formt 4 Zahlen haben
# '\s\D*\s\w*' = Hochkomma, Whitespace, Nondigit, whitespace, wordcharacters, hochkomme


In [ ]:
#Fehler mit re.findall rausfiltern in eigene Liste
gemeinde_errors = [re.findall(r"""'\s\D*\s\w*'""", i) for i in gemeinde]

In [ ]:
[re.findall(r"""'\s\D*\s\w*'""", i) for i in gemeinde]

In [ ]:
gemeinde_errors

In [ ]:
#mit Regex Fehler in Liste gemeinde entfernen und in neue Liste
gemeinde_new = [re.sub(r'\W\sAbtwil\s\W', '9030 Abtwil', i) for i in gemeinde]

In [ ]:
test = [re.findall('\W\sAbtwil\s\W', i) for i in gemeinde]

In [ ]:
test
#funktioniert nicht, Fehler sind zudem sehr unterschiedlich, also auch nicht gleich strukturiert
#-> folglich daten putzen im Excel mit Hilfe der dortigen Filtermöglichkeiten

In [ ]:
' Abtwil ' in gemeinde

In [ ]:
gemeinde_new

In [ ]:
#überprüfen, ob der Fehler entfernt worden ist
gemeinde_errorsTest = [re.findall("\s\D*\s", i) for i in gemeinde_new]
gemeinde_errorsTest